In [4]:
import numpy as np
import cv2
from pycocotools.coco import COCO

# Ruta al archivo de anotaciones COCO
ann_file_test = './dataset/Proyecto-SIS330.v12i.coco-segmentation/test/_annotations.coco.json'
ann_file_train = './dataset/Proyecto-SIS330.v12i.coco-segmentation/train/_annotations.coco.json'
ann_file_val = './dataset/Proyecto-SIS330.v12i.coco-segmentation/valid/_annotations.coco.json'

# Inicializar el objeto COCO con el archivo de anotaciones
coco_test = COCO(ann_file_test)
coco_train = COCO(ann_file_train)
coco_val = COCO(ann_file_val)
#print("-------------------")
#print(coco.info())
#print("-------------------")

# Obtener los identificadores de las imágenes en el conjunto de datos
img_ids_test = coco_test.getImgIds()
print("Cant. images test: ", len(img_ids_test))
img_ids_train = coco_train.getImgIds()
print("Cant. images train: ", len(img_ids_train))
img_ids_val = coco_val.getImgIds()
print("Cant. images valid: ", len(img_ids_val))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Cant. images test:  103
Cant. images train:  5292
Cant. images valid:  502


In [5]:
def create_one_hot_mask(coco, img_id, num_classes):
    img_info = coco.loadImgs(img_id)[0]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)
    a, b, c = img_info['file_name'].split('.')[:3]
    name_mask = f"{a}.{b}.{c}.png"
    
    # Crear una lista de máscaras binarias para cada clase (incluyendo el fondo)
    masks = [np.zeros((img_info['height'], img_info['width']), dtype=np.uint8) for _ in range(num_classes)]
    
    # Rellenar las máscaras binarias con las regiones de interés de las anotaciones COCO
    for annotation in annotations:
        segmentations = annotation['segmentation']
        class_id = annotation['category_id']  # Identificador de clase
        
        for segmentation in segmentations:
            # Desempaquetar las tuplas de las coordenadas de segmentación
            points = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
            
            # Convertir la segmentación en un polígono y asignar 1 a las regiones de interés en la máscara
            cv2.fillPoly(masks[class_id - 1], [points], 1) # type:ignore class_id - 1 para ajustar el índice (comenzando desde 0)
    
    # Concatenar las máscaras en un solo array tridimensional (one-hot encoding)
    mask = np.stack(masks, axis=-1)
    
    # Agregar el fondo como la cuarta clase en el one-hot encoding (todos los canales son 0 para el fondo)
    mask[..., num_classes - 1] = np.logical_not(mask.any(axis=-1)).astype(np.uint8)
    
    return mask, name_mask

In [6]:
def generate_masks(num_classes):
    # crear mascara para test
    for img_id in img_ids_test:
        mask, name_mask = create_one_hot_mask(coco_test, img_id, num_classes)
        cv2.imwrite('./Dataset/Proyecto-SIS330.v12i.coco-segmentation/testmasks/' + name_mask, mask*255)
    # crear mascara para train
    for img_id in img_ids_train:
        mask, name_mask = create_one_hot_mask(coco_train, img_id, num_classes)
        cv2.imwrite('./Dataset/Proyecto-SIS330.v12i.coco-segmentation/trainmasks/' + name_mask, mask*255)
    # crear mascara para valid
    for img_id in img_ids_val:
        mask, name_mask = create_one_hot_mask(coco_val, img_id, num_classes)
        cv2.imwrite('./Dataset/Proyecto-SIS330.v12i.coco-segmentation/validmasks/' + name_mask, mask*255)

In [7]:
generate_masks(num_classes=3)
print("Done")

Done
